In [38]:
import os
import torch
import dask.dataframe as dd
import json
import ast

In [23]:
ddf = dd.read_json('dummy.jsonl', lines=True)
ddf = ddf.repartition(npartitions=3)

In [24]:
print(ddf)

Dask DataFrame Structure:
                    k    accl
npartitions=3                
               string  string
                  ...     ...
                  ...     ...
                  ...     ...
Dask Name: repartition, 4 graph layers


In [27]:
def partition_to_tensor(partition):
    # Convert Dask partition (Pandas DataFrame) to NumPy array
    data = partition.computed().to_numpy()
    # Convert NumPy array to PyTorch tensor
    return torch.tensor(data, dtype=torch.float32)

In [39]:
for partition in ddf.to_delayed():
    df = partition.compute()
    print(ast.literal_eval(df['accl'][0]))
    break


[[1.1, 2.2], [1.1, 2.2], [1.1, 2.2], [1.1, 2.2]]


In [25]:
folders_list = ['./data1', './data2', './data3']

for folder in folders_list:
    for file in os.listdir(folder):
        filename = os.path.join(folder, file)
        print(f'filename: {filename}')
        with open(filename, 'r') as file:
            for line_number, line in enumerate(file):
                data = json.loads(line)
                # print(len(data['accl']))
                for point in data['accl']:
                    for index, value in enumerate(point):
                        if isinstance(value, str):
                            print(f'line_number: {line_number}, index: {index}, value: {value}')

filename: ./data1\data_part0.jsonl
filename: ./data1\data_part1.jsonl
filename: ./data2\data_part0.jsonl
filename: ./data2\data_part1.jsonl
filename: ./data3\data_part0.jsonl
filename: ./data3\data_part1.jsonl


In [7]:
import ast
def convert_to_list(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

data1 = dd.read_json('rough.jsonl', lines=True, converters={'accl': convert_to_list})

print("-------------------------------------------")

print(data1.head)


# pandas_df = data.compute()
# print(pandas_df)

TypeError: An error occurred while calling the read_json method registered to the pandas backend.
Original Message: read_json() got an unexpected keyword argument 'convertors'

In [4]:
import ast
x = "[[1.1,2],[3,4],[5,6]]"
y = ast.literal_eval(x)
print(y)
print(type(y))
print(type(y[0][0]))

[[1.1, 2], [3, 4], [5, 6]]
<class 'list'>
<class 'float'>


In [86]:
# print(data1)
# print((data1.loc[1, 'accl'].compute()))
# Define a custom function to convert strings to floats

# def convert_to_float(series):
#     print("-------------------------------------------")
#     print(series)
#     return ast.literal_eval(series)

# Apply the function to the 'accl' column
data1['accl'] = data1['accl'].map_partitions(convert_to_float)

# s = data1['accl'][0]
# print(type(s))
# print(ast.literal_eval())

<class 'dask.dataframe.core.Series'>


TypeError: literal_eval() missing 1 required positional argument: 'node_or_string'

In [32]:


# Define a custom function to check if a value is a string
def is_string(series):
    return series.apply(lambda x: isinstance(x, str))

# Apply the function to the desired column (e.g., 'column_name')
result = data['accl'].map_partitions(is_string)

# Compute the result to get the output
print(result.compute())

0    True
1    True
2    True
Name: accl, dtype: bool


In [45]:
cell = data.loc[1, 'accl'].compute()
with open('text.txt', 'w') as file:
    for point in cell:
        file.write(f'{point}\n')
